# Building a Database of NBA Games, Teams, Players, and Box Scores

A box score is a tabular representation of everything that happened during a professional basketball game in the NBA. For example, a box score for a recent game that took place between the Cleveland Cavaliers and the Milwaukee Bucks may be found at https://www.espn.com/nba/boxscore/_/gameId/401360541.

For this lab we will be working with data that collects and appends all box scores from all NBA games together. The data were compiled by Stewart Gibson of Advanced Sports Analytics.

The raw data has 81 columns. Below is a data dictionary.

In [1]:
import pandas as pd
pd.read_csv(filepath_or_buffer = 'Dictionary--Box_Scores.csv', sep = '\t', index_col = 'Column No.')

,Column name,Data type,Varies By,Calculated from other columns?,Description
Column No.,,,,,
0,game_id,object,Game,No,Unique ID for one NBA game
1,game_date,object,Game,No,Date in YYYY-MM-DD format
2,OT,int64,Game,No,Boolean: did the game go into overtime?
3,H_A,object,Game and Team,No,Is the team playing at home or away in this game?
4,Team_Abbrev,object,Team,No,Team
...,...,...,...,...,...
76,SG%,float64,Game and Player,No,% of player's minutes playing the point guard ...
77,SF%,float64,Game and Player,No,% of player's minutes playing the point guard ...
78,PF%,float64,Game and Player,No,% of player's minutes playing the power forwar...


## Goals

1. Construct a series of data tables which together form a database in third normal form. That will require us to create separate tables for games, teams, players, and box score statistics. Five tables:

    a. Information overall about the game: OT, date, location, etc

    b. Info about how the team overall did in the game

    c. Info about how the player did personally in the game

    d. Info about the team's total season stats so far

    e. Info about the player's total season so far

    Use the aggregation methods in pandas to construct these tables.

Create an ER diagram.

Initialize a local database.

Upload the data tables to the database.

Issue SQL queries to the database.

## We load all box scores from all NBA games.

In [2]:
import numpy as np
import pandas as pd
import psycopg2
import dotenv
import os
from sqlalchemy import create_engine

In [3]:
nba = pd.read_csv('ASA All NBA Raw Data.csv')
nba.head(n = 5)

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537


In [4]:
pd.options.display.max_rows = None
nba.head(1).T.head(n = 5)

,0
game_id,202202170BRK
game_date,2022-02-17
OT,0
H_A,A
Team_Abbrev,WAS


## We drop column Inactives to ensure first-normal form.

In [5]:
pd.options.display.max_rows = 10
nba = nba.drop('Inactives', axis=1)
nba.head(1).T.head(n = 5)

,0
game_id,202202170BRK
game_date,2022-02-17
OT,0
H_A,A
Team_Abbrev,WAS


## We filter the data to just the current NBA season

In [6]:
nba2022 = nba.query("season==2022")
nba2022

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,2022-02-17,0,A,WAS,117,94.5,0.627,13.5,22.9,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22619,202201130NOP,2022-01-13,0,A,LAC,89,97.0,0.444,14.3,10.2,...,0.451128,1.00,7.977756,4.313333,0.0,0.0,0.0,0.0,100.0,48.215347
22620,202201150SAS,2022-01-15,0,A,LAC,94,92.7,0.440,11.9,35.0,...,NaN,0.00,7.997933,4.324242,0.0,0.0,0.0,0.0,100.0,57.995905
22621,202112220SAC,2021-12-22,0,A,LAC,105,95.5,0.555,11.9,15.4,...,0.000000,0.00,NaN,NaN,0.0,0.0,0.0,0.0,100.0,NaN
22622,202112260LAC,2021-12-26,0,H,LAC,100,99.8,0.512,13.0,17.4,...,NaN,0.00,2.712684,1.466667,0.0,0.0,0.0,0.0,100.0,41.567992


## Information overall about the game: OT, date, location, etc

In [25]:
games = nba2022[['game_id', 'game_date', 'OT', 'season']].drop_duplicates()
games.columns = [x.lower() for x in games.columns]
games

,game_id,game_date,ot,season
0,202202170BRK,2022-02-17,0,2022
26,202202170CHO,2022-02-17,2,2022
48,202202170LAC,2022-02-17,0,2022
71,202202170MIL,2022-02-17,0,2022
95,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
13978,202111190NOP,2021-11-19,0,2022
13984,202111290LAC,2021-11-29,0,2022
14001,202201130NOP,2022-01-13,0,2022
14005,202201250PHI,2022-01-25,0,2022


In [26]:
players = nba[['player_id', 'player']].drop_duplicates()
players.columns = [x.lower() for x in players.columns]
players

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
...,...,...
109702,frazimi01,Michael Frazier
110441,howarwi01,William Howard
110913,mbahalu01,Luc Mbah a Moute
111399,bowmaky01,Ky Bowman


## Info about how the team overall did in the game

In [24]:
nba2022['win'] = nba2022['Team_Score'] > nba2022['Opponent_Score']
team_game = nba2022[['Team_Abbrev', 'H_A', 'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg']].drop_duplicates()
team_game.columns = [x.lower() for x in team_game.columns]
team_game

,team_abbrev,h_a,team_score,team_pace,team_efg_pct,team_tov_pct,team_orb_pct,team_ft_rate,team_off_rtg
0,WAS,A,117,94.5,0.627,13.5,22.9,0.157,123.8
13,BRK,H,103,94.5,0.483,13.1,33.3,0.191,109.0
26,MIA,A,111,88.8,0.471,11.1,26.8,0.147,103.4
37,CHO,H,107,88.8,0.453,13.6,28.1,0.221,99.7
48,HOU,A,111,103.7,0.533,15.3,24.0,0.154,107.1
...,...,...,...,...,...,...,...,...,...
15181,LAC,A,116,98.6,0.547,8.1,20.4,0.115,117.6
15237,LAC,A,105,95.5,0.555,11.9,15.4,0.171,110.0
15238,LAC,H,100,99.8,0.512,13.0,17.4,0.140,100.2
15239,LAC,H,108,99.5,0.454,9.2,29.8,0.206,108.6


## Info about how the player did personally in the game

In [23]:
player_game = nba2022[['game_id', 'player_id', 'Opponent_Abbrev', 'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'player', 'starter', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm', 'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute', 'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting', 'last_60_minutes_per_game_bench', 'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'active_position_minutes']]
player_game.columns = [x.lower() for x in player_game.columns]
player_game

,game_id,player_id,opponent_abbrev,opponent_score,opponent_pace,opponent_efg_pct,opponent_tov_pct,opponent_orb_pct,opponent_ft_rate,opponent_off_rtg,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,pg%,sg%,sf%,pf%,c%,active_position_minutes
0,202202170BRK,kispeco01,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,kuzmaky01,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,caldwke01,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,netora01,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,bryanth01,BRK,103,94.5,0.483,13.1,33.3,0.191,109.0,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22619,202201130NOP,gabriwe01,NOP,113,97.0,0.515,5.5,24.1,0.134,116.5,...,0.451128,1.00,7.977756,4.313333,0.0,0.0,0.0,0.0,100.0,48.215347
22620,202201150SAS,gabriwe01,SAS,101,92.7,0.476,12.9,31.3,0.265,109.0,...,NaN,0.00,7.997933,4.324242,0.0,0.0,0.0,0.0,100.0,57.995905
22621,202112220SAC,wrighmo01,SAC,89,95.5,0.471,15.1,15.9,0.106,93.2,...,0.000000,0.00,NaN,NaN,0.0,0.0,0.0,0.0,100.0,NaN
22622,202112260LAC,wrighmo01,DEN,103,99.8,0.489,12.3,20.8,0.105,103.2,...,NaN,0.00,2.712684,1.466667,0.0,0.0,0.0,0.0,100.0,41.567992


## Info about the team's total season stats so far

In [ ]:
teams = nba2022[['Team_Abbrev', 'game_id', 'fg', 'fga', 'fg3', 'fg3a', 
             'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']]
teams = teams.groupby('Team_Abbrev').agg({'fg': sum,
                                          'fga': sum, 
                                          'fg3': sum, 
                                          'fg3a': sum, 
                                          'ft': sum, 
                                          'fta': sum, 
                                          'orb': sum, 
                                          'drb': sum, 
                                          'ast': sum, 
                                          'stl': sum, 
                                          'blk': sum, 
                                          'tov': sum, 
                                          'pf': sum, 
                                          'pts': sum})
WL = nba2022.groupby(['Team_Abbrev', 'game_id']).agg({'win': 'mean'})
WL = WL.groupby('Team_Abbrev').agg({'win':[sum, 'count']})
WL.columns = ['wins', 'totalgames']
WL['losses'] = WL['totalgames'] - WL['wins']
teams = pd.merge(teams, WL, on=['Team_Abbrev'], validate='one_to_one')
teams = teams.reset_index()
teams

## Info about the player's total season so far

In [ ]:
players = nba2022[['player', 'player_id', 'starter', 'minutes', 'fg', 'fga', 'fg3', 'fg3a', 
                   'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'usg_pct', 
                   'did_not_play', 'is_inactive', 'ts_pct', 'PG%', 'SG%', 'SF%', 'PF%', 'C%']]
players = players.groupby(['player','player_id']).agg({'starter': sum,
                                                   'minutes': 'mean',
                                                   'fg': sum,
                                                   'fga': sum, 
                                                   'fg3': sum, 
                                                   'fg3a': sum, 
                                                   'ft': sum, 
                                                   'fta': sum, 
                                                   'orb': sum, 
                                                   'drb': sum, 
                                                   'ast': sum, 
                                                   'stl': sum, 
                                                   'blk': sum, 
                                                   'tov': sum, 
                                                   'pf': sum, 
                                                   'pts': sum,
                                                  'usg_pct': 'mean',
                                                   'ts_pct': 'mean',
                                                  'did_not_play': sum,
                                                  'is_inactive': sum,
                                                  'PG%': 'mean', 
                                                   'SG%': 'mean', 
                                                   'SF%': 'mean', 
                                                   'PF%': 'mean', 
                                                   'C%': 'mean'}).add_prefix('season_')
players.reset_index()

# PostgreSQL

In [30]:
dotenv.load_dotenv()
pgpassword = os.getenv("pgpassword")

In [31]:
dbserver = psycopg2.connect(
    user = 'tlever',
    password = pgpassword,
    host = 'localhost'
)
dbserver.autocommit = True

In [32]:
cursor = dbserver.cursor()

In [33]:
fix = '''
SELECT pg_terminate_backend(pg_stat_activity.pid)
FROM pg_stat_activity
WHERE pg_stat_activity.datname = 'nbadb';
'''
cursor.execute(fix)

In [34]:
try:
    cursor.execute('CREATE DATABASE nbadb')
except:
    cursor.execute('DROP DATABASE nbadb')
    cursor.execute('CREATE DATABASE nbadb')

In [35]:
engine = create_engine(
    'postgresql+psycopg2://{user}:{pw}@{host}/{db}'
    .format(user = 'tlever', pw = pgpassword, host = 'localhost', db = 'nbadb')
)

In [36]:
player_game.to_sql('player_game', con = engine, index = False, chunksize = 1000, if_exists = 'replace')
team_game.to_sql('team_game', con = engine, index = False, chunksize = 1000, if_exists = 'replace')
players.to_sql('players', con = engine, index = False, chunksize = 1000, if_exists = 'replace')
game_info.to_sql('game_info', con = engine, index = False, chunksize = 1000, if_exists = 'replace')

883

In [37]:
myquery = '''
SELECT * FROM team_game WHERE team_abbrev='CLE'
'''

In [38]:
pd.read_sql_query(myquery, con = engine)

,team_abbrev,h_a,team_score,team_pace,team_efg_pct,team_tov_pct,team_orb_pct,team_ft_rate,team_off_rtg
0,CLE,A,116,91.8,0.602,12.2,33.3,0.114,126.4
1,CLE,A,93,90.5,0.519,15.2,13.5,0.169,102.8
2,CLE,A,102,93.6,0.478,10.8,28.8,0.178,109.0
3,CLE,A,120,100.7,0.586,12.1,17.1,0.309,119.1
4,CLE,H,105,96.1,0.530,12.3,24.4,0.220,109.2
...,...,...,...,...,...,...,...,...,...
53,CLE,H,115,101.7,0.505,8.5,26.0,0.175,113.0
54,CLE,H,124,96.4,0.675,14.2,22.5,0.200,128.7
55,CLE,A,119,96.6,0.589,11.9,22.5,0.144,123.2
56,CLE,A,101,99.3,0.440,12.8,29.4,0.217,101.7
